# Домашнее задание к лекции Работа с файловой системой и модули

## 1. Переведите содержимое файла purchase_log.txt в словарь purchases вида:
{‘1840e0b9d4’: ‘Продукты’, …}

In [34]:
# Т.к. при попытке вывести на экран весь словарь purchases выдается ошибка:
#            IOPub data rate exceeded.
#            The notebook server will temporarily stop sending output
#            to the client in order to avoid crashing it.
#            To change this limit, set the config variable
#            `--NotebookApp.iopub_data_rate_limit`.

#            Current values:
#            NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
#            NotebookApp.rate_limit_window=3.0 (secs)
# для проверки работы кода в этом случае ограничил размер словаря 10-ю элементами

import json
purchases = {}
with open('purchase_log.txt', 'r', encoding="utf-8") as f:
    for i, line in enumerate(f):
        if 0 < i < 11:
            dict_ = json.loads(line)
            purchases[dict_['user_id']] = dict_['category']
print(purchases)



{'1840e0b9d4': 'Продукты', '4e4f90fcfb': 'Электроника', 'afea8d72fc': 'Электроника', '373a6055fe': 'Бытовая техника', '9b2ab046f3': 'Электроника', '9f39d307c3': 'Электроника', '44edeffc91': 'Продукты', '704474fa2d': 'Продукты', '1de31be403': 'Бытовая техника', 'b71f36a5e4': 'Продукты'}


In [28]:
# В этом случае словарь формируется полностью. Ограничен вывод элементов словаря на экран.
# Если в файле purchase_log.txt какой-то user_id встречается несколько раз, то в итоговом словаре purchases
# для этого ключа user_id в список добавляются все связанные с ним категории покупок.

import json
purchases = {}           # словарь, где для каждого user_id категории покупок не повторяются
purchases_w_duble = {}   # в этот словарь для каждого user_id записываются все встречающиеся категории, даже если повторяются
with open('purchase_log.txt', 'r', encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i > 0:
            dict_ = json.loads(line)
            if dict_['user_id'] in purchases:
                purchases_w_duble[dict_['user_id']].append(dict_['category'])
                if dict_['category'] not in purchases[dict_['user_id']]:
                    purchases[dict_['user_id']].append(dict_['category'])
            else:
                purchases[dict_['user_id']] = [dict_['category']]
                purchases_w_duble[dict_['user_id']] = [dict_['category']]
# Выводим на экран первые несколько элементов
i = 0
for key, cat in purchases.items():
    print(key, cat)
    i += 1
    if i > 10:
        break
# записываем результат в файл purchase_result.txt
with open('purchase_result.txt', 'w') as w:
    w.write(str(purchases))
#     w.write(json.dumps(purchases))

1840e0b9d4 ['Продукты']
4e4f90fcfb ['Электроника']
afea8d72fc ['Электроника']
373a6055fe ['Бытовая техника']
9b2ab046f3 ['Электроника']
9f39d307c3 ['Электроника']
44edeffc91 ['Продукты']
704474fa2d ['Продукты']
1de31be403 ['Бытовая техника']
b71f36a5e4 ['Продукты']
79843a685a ['Продукты']


## 2. Для каждого user_id в файле visit_log.csv определите третий столбец с категорией покупки (если покупка была, сам файл visit_log.csv изменять не надо). Запишите в файл funnel.csv визиты из файла visit_log.csv, в которых были покупки с указанием категории.

In [50]:
# Некоторые user_id в файле purchase_log.txt встречаются несколько раз, но при этом в файле visit_log.csv встречаются только 
# один раз. Т.е. возможно покупатель за один визит совершил покупки в нескольких категориях (или несколько покупок в одной).
# А некотоные user_id, встречающиеся в обоих файлах по несколько раз (например user_id = 0061aaf3f9). Каким образом разделить
# в какой визит покупатель покупал в конкретной категории я не знаю, не хватает входных данных. Поэтому для каждого визита 
# буду указывать весь список категорий покупок этого покупателя (включая дубли).
# Формат итогового файла funnel.csv такой же, как visit_log.csv

with open('visit_log.csv', 'r') as visit_log, open('funnel.csv', 'w') as funnel, open('visit_wo_pur.csv', 'w') as visit_wo_pur:
    funnel.write('user_id,source,category\n')
    for i, visit_line in enumerate(visit_log):
        visit_line = visit_line.strip().split(',')
        if visit_line[0] in purchases_w_duble:
            visit_line.append(','.join(purchases_w_duble[visit_line[0]]))
            stroka = ','.join(visit_line)+'\n'
            funnel.write(stroka)
        else:
            stroka = ','.join(visit_line)+'\n'
            visit_wo_pur.write(stroka)
        
#  В файле с визитами visit_log.csv 200000 строк
#  В файле funnel.csv 13799 строк (одну строку с заголовками мы добавили вручную. Без нее 13798 строк)
#  В файл visit_wo_pur.csv попало то, что не нашли в словаре с покупками, т.е. визиты без покупок и заголовок 
#  из visit_log.csv. В этом файле 186202 строки. 
#    13798 + 186202 = 200000 - вроде бы все сходится, ничего не потеряли
